# СРС №2. Обслуживание простых деревьев кодом. Список соседства
## Задание.
1. Реализовать структуру таблиц, реализовывающую структуру для хранения деревьев «Список соседства» и позволяющую хранить данные выбранной тематики. Тематика может быть выбрана из списка предложенных (см. Предлагаемые темы) или предложена студентом.
2. Наполнить данными реализованную структуру. Наполнение и генерация данных должны производиться с программным кодом. Таблица с деревом должна содержать не менее 150 строк, каждый узел, не являющийся листом должен иметь от 2 до 5 потомков, общая глубина дерева должна быть не менее 10.
3. Реализовать следующие операции в виде функций программной бибилотеки с текстовым или графическим интерфейсом:
- добавление листа;
- удаление листа;
- удаление поддерева;
- перемещение листа;
- перемещение поддерева;
- удаление узла без поддерева;
- перемещение узла без поддерева;
- получение прямых потомков;
- получение прямого родителя;
- получение всех потомков (с сохранением иерархичности);
- получение всех родителей (с сохранением иерархичности);

In [1]:
import mysql.connector
import pandas as pd
import random 
from faker import Faker
import pandas as pd
from mysql.connector import connect, Error
#import getpass
fake = Faker('ru_RU')
r_host = "localhost"
r_user = "root"
r_password = "aryuna"
r_db = "srs2"
connection = connect(
    host=r_host, 
    user=r_user,
    password=r_password,
    database = r_db,
)
db_cur = connection.cursor()
from queue import Queue

In [2]:
def AddChildren(ancestors):
    generation = []
    while not ancestors.empty():
        descend = random.randint(2, 5) #сколько детей будет у текущего
        new_parent = ancestors.get()
        for i in range(descend): #по очереди создаём
            #print(f"{i} по идее, какой сейчас по счёту ребенок, а {new_parent} - родитель. Детей должно быть {descend}")
            db_cur.execute(f'INSERT INTO AdjacencyList (name, parent_id) VALUES ("{fake.company()}", {new_parent})')
            connection.commit()
        db_cur.execute(f"SELECT id FROM AdjacencyList WHERE parent_id = {new_parent}")
        generation.append(db_cur.fetchall())
    this_gen_par = []
    for _list in generation: #списки развернуть и сделать не список списков, а просто список
        for x in _list:
            this_gen_par.append(int(x[0])) # x[0] чтоб избавиться от кортежа
    #print(f'\t {this_gen_par}')
    return this_gen_par

def NewParents(ancestors, gen):
#сначала достаём всё нынешнее поколение
#потом генерим кол-во детей, которые родят
#а потом рандомсэмплим тех, кто родит и всех их кладём в ancestors
    chil_num = random.randint(1, len(gen)) #сначала генерим то, сколько из детей станет родителями
    gen = random.sample(gen, chil_num) #получаем тех, кто станет родителями
    for x in gen:
        ancestors.put(x) #кладём в очередь

In [3]:
def TreeGenerator(d):
    ancestors = Queue() #очередь родителей
    #создание корня, id будет 1 из-за автоинкремента
    db_cur.execute(f'INSERT INTO AdjacencyList (name, parent_id) VALUES ("{fake.company()}", NULL)')
    ancestors.put(1)
    new_generation = AddChildren(ancestors) #получение нынешнего поколения
    #print(ancestors.qsize()) #0 после 1-го раза - потому что в процессе добавления детей всё удалилось
    depth = 1 #Т.к. уже случился AddChildren 
    while (depth < d):
        NewParents(ancestors, new_generation)
        new_generation = AddChildren(ancestors)
        depth += 1

In [4]:
def GetParent(v):
    db_cur.execute(f"SELECT parent_id FROM AdjacencyList WHERE id = {v}")
    return db_cur.fetchone()[0]
def GetChildren(v):
    db_cur.execute(f"SELECT id FROM AdjacencyList WHERE parent_id = {v}")
    children = [x[0] for x in db_cur.fetchall()]
    return children
def GetAllParents(v):
    db_cur.execute(f"""WITH RECURSIVE parents AS (
    SELECT id, name, parent_id FROM adjacencylist where id = {v}
    UNION ALL 
    SELECT aj.id, aj.name, aj.parent_id FROM adjacencylist aj JOIN parents ON aj.id = parents.parent_id
    )
    SELECT * FROM parents group by id""")
    for i in db_cur:
        print(i)
def GetAllChildren(v):
    db_cur.execute(f"""WITH recursive children as (
    SELECT id, name, parent_id FROM AdjacencyList where parent_id = {v}
    union all
    select aj.id, aj.name, par.id FROM AdjacencyList aj JOIN children par ON par.id = aj.parent_id
    )
    SELECT * from children group by id
    """)
    for i in db_cur:
        print(i)

In [5]:
def AddLeaf(title, parent):
    db_cur.execute(f'INSERT INTO AdjacencyList (name, parent_id) VALUES ("{title}", {parent})')
    connection.commit()
def DelSubTreeOrLeaf(_id):
    db_cur.execute(f"DELETE FROM AdjacencyList WHERE id = {_id}") #удаление с помощью внешнего ключа
    connection.commit()
def MoveSubTreeOrLeaf(v, change):
    db_cur.execute(f"UPDATE AdjacencyList SET parent_id = {change} WHERE id = {v}")
    connection.commit()
def DelNodeWithoutTree(node):
    x = GetParent(node)
    db_cur.execute(f"""UPDATE adjacencylist a1 JOIN adjacencylist a2 ON a2.id = {node}
SET a1.parent_id = a2.parent_id WHERE a1.parent_id = {node}""")
    connection.commit()
    #в лекции было с подзапросом select, но нельзя обновить таблицу, в которой происходит чтение
    db_cur.execute(f"DELETE FROM AdjacencyList WHERE id = {node}")
    connection.commit()
def MoveNodeWithoutTree(node, change):
#получаем родителя, потом детей, которых переставляем к родителю
    db_cur.execute(f"""UPDATE adjacencylist a1 JOIN adjacencylist a2 ON a2.id = {node}
SET a1.parent_id = a2.parent_id WHERE a1.parent_id = {node}""")
    connection.commit()
    MoveSubTreeOrLeaf(node, change)

In [6]:
def GetDepth():
    db_cur.execute("""WITH RECURSIVE parents AS ( 
    SELECT id, parent_id FROM adjacencylist
    UNION ALL 
    SELECT parents.id, aj.parent_id FROM adjacencylist aj JOIN parents ON aj.id = parents.parent_id
    AND NOT ISNULL(aj.parent_id)
    )
    SELECT MAX(depth) FROM (SELECT count(id) as depth FROM parents group by id) d;
    """)
    return db_cur.fetchone()[0]
def CheckChildren():
    db_cur.execute("""WITH recursive children as (
    SELECT id, name, parent_id FROM AdjacencyList where parent_id = 1
    union all
    select aj.id, aj.name, par.id FROM AdjacencyList aj JOIN children par ON par.id = aj.parent_id
    )
    select parent_id, count(id) from children group by parent_id
    """)
    count_children = [x[1] for x in db_cur.fetchall()]
    ma = max(count_children)
    mi = min(count_children)
    mess = 0 if mi >= 2 and ma <= 5 else f'...КОД КРАСНЫЙ!!! ГДЕ-ТО ДЕТЕЙ {mi}, ГДЕ-ТО {ma} - ЧТО-ТО ПОШЛО НЕ ТАК!!!'
    if isinstance(mess, str):
        print(mess)
        print('Код принят. Сворачиваем работу.')
        return False
    return True
def CheckWithJoins():
    print("МИЛЛИОН МИЛЛИОН ДЖОИНОООВ")
    db_cur.execute(""" SELECT distinct D10.parent_id, d9.parent_id, d8.parent_id, d7.parent_id, d6.parent_id, d5.parent_id, d4.parent_id,
 d3.parent_id, d2.parent_id, d1.parent_id, d0.id
-- SELECT distinct d10.parent_id
from adjacencylist d1 JOIN adjacencylist D0 ON D0.id = D1.parent_id AND ISNULL(D0.parent_id)
JOIN adjacencylist d2 ON d1.id = d2.parent_id JOIN adjacencylist D3 ON D2.id = d3.parent_id 
JOIN adjacencylist d4 ON D3.id = d4.parent_id JOIN adjacencylist d5 ON D4.id = d5.parent_id 
JOIN adjacencylist d6 ON D5.id = d6.parent_id JOIN adjacencylist d7 ON D6.id = d7.parent_id 
JOIN adjacencylist d8 ON D7.id = d8.parent_id JOIN adjacencylist d9 ON D8.id = d9.parent_id 
JOIN adjacencylist d10 ON D9.id = d10.parent_id
ORDER BY 1, 2, 3, 4, 5, 6, 7, 8, 9, 10;""")
    for i in db_cur:
        print(i)

In [7]:
drop = ("DROP TABLE IF EXISTS AdjacencyList")
adj = ("""CREATE TABLE AdjacencyList(
 id INT PRIMARY KEY AUTO_INCREMENT,
 name VARCHAR(100),
 parent_id INT,
 FOREIGN KEY(parent_id) REFERENCES AdjacencyList(id) ON DELETE CASCADE
)
""") 
db_cur.execute(drop)
db_cur.execute(adj)
connection.commit()
d = random.randint(10, 15)
TreeGenerator(d)

In [8]:
def FitTree(ne):
    db_cur.execute("""WITH recursive children as (
    SELECT id, name, parent_id FROM AdjacencyList WHERE parent_id = 1
    union all
    select aj.id, aj.name, par.id FROM AdjacencyList aj JOIN children par ON par.id = aj.parent_id
    )
    select parent_id, count(id) from children group by parent_id
    """)
    parents = [[x[0], x[1]] for x in db_cur.fetchall()]
    for i in range(ne + int(ne/2)):
        if random.choice(parents)[1] < 5:
            db_cur.execute(f'INSERT INTO AdjacencyList (name, parent_id) VALUES ("{fake.company()}", {parents[i][0]})')
            connection.commit()
def Checking():
    db_cur.execute("SELECT COUNT(id) FROM adjacencylist")
    n = db_cur.fetchone()[0]
    print(f"Количество элементов - {n}")
    if n < 150:
        FitTree(150 - n)
    print(f"NOW Количество элементов - {n}")
    print(f"Глубина дерева - {GetDepth()}")
    print("""Если где-то детей будет не доставать или их будет слишком много, выведется сообщение и программа остановится. 
    В противном случае вас ожидает гордое молчание.""")
    return CheckChildren()
    #CheckWithJoins()

In [9]:
def Interface():
    tab = ''
    a = input("Введите вершину для махинаций. Если понадобится что-то ещё, введёте потом. ")
    while tab != 'Всё':
        db_cur.execute(f"SELECT id FROM adjacencylist where id = {a}")
        check = db_cur.fetchone()
        if check is None:
            a = input("Этой вершины нет, давай по новой: ")
            continue
        tab = input("Введите число от 1 до 9. Или 0: ")
        if tab == '1':
            print(f"Родитель вершины {a}: {GetParent(a)}")
        elif tab == '2':
            print(f"Прямые потомки вершины {a}: {GetChildren(a)}")
        elif tab == '3':
            name = input(f"В вершину {a} добавляется лист. Введите его имя: ")
            AddLeaf(name, a)
        elif tab == '4':
            print(f'Удаляется вершина {a} (с поддеревом, если оно есть). ')
            DelSubTreeOrLeaf(a)
        elif tab == '5':
            new = input(f"Куда переместить вершину {a} (с поддеревом, если оно есть)? ")
            MoveSubTreeOrLeaf(a, new)
        elif tab == '6':
            print(f'Удаляется вершина {a} (без поддерева). ')
            DelNodeWithoutTree(a)
        elif tab == '7':
            new = input(f"Куда переместить вершину {a} (без поддерева)? ")
            MoveNodeWithoutTree(a, new)
        elif tab == '8':
            print(f"Получение ВСЕХ РОДИТЕЛЕЙ до {a}!!! ")
            GetAllParents(a)
        elif tab == '9':
            print(f"Получение ВСЕХ ПОТОМКОВ {a}!!! ")
            GetAllChildren(a)
        elif tab == '0':
            a = input("Новая вершина для махинаций: ")
        elif tab == 'Проверка':
            if Checking() is not True:
                break
            #CheckWithJoins()
        elif tab == "Всё":
            print("Oк. ")
        else:
            print("Вы ввели что-то не то... ")
    depth = 0
    print("До свидания.")

# ы
    1 - прямой родитель
    2 - прямые потомки
    3 - добавление листа
    4 - удаление листа/поддерева
    5 - перемещение листа/поддерева
    6 - удаление узла без поддерева
    7 - перемещение узла без поддерева
    8 - получение всех потомков (с сохранением иерархичности)
    9 - получение всех родителей (с сохранением иерархичности)
    0 - выбор другой вершины

In [ ]:
Interface()

def DelSubTree(v): (Т_Т)
    children = GetChildren(v)
    for i in children:
        DelSubTree(i)
    db_cur.execute(f"DELETE FROM AdjacencyList WHERE id = {v}")
    # удаление с помощью внешнего ключа
    connection.commit()

depth = 0
def Depth(last):
    global depth
    t = GetParent(last)
    if GetParent(t):
        depth += 1
        Depth(t)
    return depth

def Depth(last, depth):
    t = GetParent(last)
    if GetParent(t):
        depth += 1
        Depth(t, depth)
    print(depth)
    return depth

In [10]:
Checking()

Количество элементов - 858
NOW Количество элементов - 858
Глубина дерева - 15
Если где-то детей будет не доставать или их будет слишком много, выведется сообщение и программа остановится. 
    В противном случае вас ожидает гордое молчание.


True

In [ ]:
#определение понятия через это самое понятие


- РЕКУРСИВНЫЙ ЗАПРОС ГЛУБИНЫ ded inside
- INNER JOIN-Ы КУЧИ-КУЧИ, ЫЫЫЫ (Т-Т) - ВАРИАНТ В ЛОБ
- ВАРИАНТ В ЛОБ - ТУПО ПРОЦЕДУРА С ПОЛУЧЕНИЕМ И СЧЁТОМ, УРАААА
- хранимая процедура Т_Т (whilы, счётчик на глубину)